<a href="https://colab.research.google.com/github/CarolinaSanchezM/Regresi-n-AcelerAd/blob/main/Copia_de_C%C3%B3digo_prueba_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Código para limpiar la base de datos y unir Inventario + Ventas

### Pasos manuales antes de ingresar tu archivo:
#### 1) Cargar un solo archivo de Excel con dos hojas llamadas 'Ventas' e 'Inventario':
##### Asegúrate de que el archivo de Excel tenga exactamente dos hojas con los nombres 'Ventas' e 'Inventario'.
##### Los nombres de las hojas deben comenzar con una letra mayúscula y coincidir exactamente con Ventas e Inventario.


#### 2)Modificar la hoja 'Ventas':
##### Eliminar la segunda columna 'Denominación':
###### Si hay dos columnas llamadas 'Denominación', elimina la segunda.
###### Ejemplo: Si tienes Denominación: AGROTANQUE PE EUREKA NEGRO 5,000 L ND y Denominación: CISTERNAS, elimina la columna con Denominación: CISTERNAS.
##### Renombrar la columna 'Total KG' a 'KG':
###### Encuentra la columna llamada 'Total KG' y renómbrala a 'KG'.
##### En la columna 'Familia' borrar los #N/A, dejar las celdas vacías.

#### 3)Modificar la hoja 'Inventario':
##### Renombrar la columna 'texto breve de material' a 'Denominación':
###### Encuentra la columna llamada texto breve de material y renómbrala a Denominación.
##### Renombrar la columna 'Valor Total Libre + Calidad' a 'Total':
###### Encuentra la columna llamada Valor Total Libre + Calidad y renómbrala a Total.

### Después de realizar estas modificaciones manuales, puedes usar el siguiente código para procesar los datos:



#### 1. Instalar la librería

In [ ]:
pip install pandas numpy

#### 2. Importar la librería

In [2]:
import pandas as pd
from google.colab import files

#### 3.Subir el archivo de excel
##### Aparece una leyenda abajo donde pondrás la ruta del archivo, pasos para copiar la ruta:

1.   Subir el archivo a google colab, de lado izquiero aparecen 5 iconos. En el icono de carpeta deberás subir el archivo de excel.
2. Click derecho al archivo que se encuentra en "Archivos" > "Copiar la ruta de acceso".
3. Pegar la ruta de acceso en la leyenda que te solicito.



In [3]:

# Cargar el archivo de Excel manualmente
file_path = input("Ingrese la ruta completa del archivo de Excel (por ejemplo, 'C:/ruta/al/archivo.xlsx'): ")

try:
    ventas_df = pd.read_excel(file_path, sheet_name='Ventas')
    inventario_df = pd.read_excel(file_path, sheet_name='Inventario')
    print("Archivo cargado exitosamente.")
except Exception as e:
    print(f"Error al leer el archivo de Excel: {e}")


Ingrese la ruta completa del archivo de Excel (por ejemplo, 'C:/ruta/al/archivo.xlsx'): /content/Base.xlsx
Archivo cargado exitosamente.


#### 4.Procesamiento de los datos
##### Aparece una leyenda donde solicitará el mes y año de interés. Posteriormente pondrás el nombre del archivo de salida (Sigue los ejemplos)

In [4]:
# Función para limpiar y sumar ventas
def limpiar_y_sumar_ventas(ventas_df):
    ventas_limpias = ventas_df[['Mes', 'Familia', 'Año', 'Centro', 'Denominación', 'Material', 'KG']]
    ventas_con_familia = ventas_limpias[ventas_limpias['Familia'].notna()]
    ventas_sin_familia = ventas_limpias[ventas_limpias['Familia'].isna()]
    ventas_agrupadas_con_familia = ventas_con_familia.groupby(
        ['Mes', 'Familia', 'Año', 'Centro', 'Denominación', 'Material'], as_index=False).agg({'KG': 'sum'})
    ventas_agrupadas_sin_familia = ventas_sin_familia.groupby(
        ['Mes', 'Año', 'Centro', 'Denominación', 'Material'], as_index=False).agg({'KG': 'sum'})
    ventas_agrupadas_sin_familia['Familia'] = 'Sin Familia'
    ventas_agrupadas = pd.concat([ventas_agrupadas_con_familia, ventas_agrupadas_sin_familia], ignore_index=True)
    ventas_agrupadas = ventas_agrupadas.rename(columns={'KG': 'Total_KG'})
    return ventas_agrupadas

# Función para unir inventario con ventas
def unir_inventario_ventas(ventas_df, inventario_df):
    inventario_agrupado = inventario_df.groupby(
        ['Centro', 'Denominación', 'Material', 'Almacén'], as_index=False).agg({'Total': 'sum'})
    ventas_df['ALPT'] = 0
    ventas_df['ALPP'] = 0
    for idx, row in inventario_agrupado.iterrows():
        mask = (
            (ventas_df['Centro'] == row['Centro']) &
            (ventas_df['Denominación'] == row['Denominación']) &
            (ventas_df['Material'] == row['Material'])
        )
        if row['Almacén'] == 'ALPT':
            ventas_df.loc[mask, 'ALPT'] += row['Total']
        elif row['Almacén'] == 'ALPP':
            ventas_df.loc[mask, 'ALPP'] += row['Total']
    return ventas_df

# Función para aplicar filtros por múltiples meses y años
def aplicar_filtro_meses_anos(ventas_df, meses, anos):
    return ventas_df[(ventas_df['Mes'].isin(meses)) & (ventas_df['Año'].isin(anos))]

# Limpiar y sumar ventas
ventas_agrupadas = limpiar_y_sumar_ventas(ventas_df)
ventas_final = unir_inventario_ventas(ventas_agrupadas, inventario_df)

# Solicitar al usuario que ingrese los meses y años para filtrar
meses_filtrados = input("Ingrese los meses que desea filtrar, separados por comas (por ejemplo, 'abril,mayo'): ").split(',')
anos_filtrados = list(map(int, input("Ingrese los años que desea filtrar, separados por comas (por ejemplo, '2023,2024'): ").split(',')))

# Aplicar el filtro por meses y años
ventas_filtradas = aplicar_filtro_meses_anos(ventas_final, meses_filtrados, anos_filtrados)

# Solicitar al usuario el nombre del archivo de salida
nombre_archivo_salida = input("Ingrese el nombre del archivo de salida (por ejemplo, 'resultado.xlsx'): ")

# Verificar si el nombre del archivo de salida tiene la extensión correcta
if not nombre_archivo_salida.endswith('.xlsx'):
    nombre_archivo_salida += '.xlsx'

# Exportar el resultado a un archivo Excel
ventas_filtradas.to_excel(nombre_archivo_salida, index=False)
print(f'Archivo guardado en {nombre_archivo_salida}')

# Descargar el archivo creado
files.download(nombre_archivo_salida)


Ingrese los meses que desea filtrar, separados por comas (por ejemplo, 'abril,mayo'): enero,febrero,marzo
Ingrese los años que desea filtrar, separados por comas (por ejemplo, '2023,2024'): 2023,2024
Ingrese el nombre del archivo de salida (por ejemplo, 'resultado.xlsx'): base_finalfinal
Archivo guardado en base_finalfinal.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#### 5. El archivo se descargará automaticamente